<a href="https://colab.research.google.com/github/Dreaming-world/learn_tensorflow_nlp/blob/master/RNN_for_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN 文本生成

In [50]:
import tensorflow as tf
import numpy as np
import os
import time

# 下载数据集

In [51]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [52]:
# 读取并为 py2 compat 解码
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# 文本长度是指文本中的字符个数
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [53]:
print(text[100])

# 建立字典

In [54]:
vocab = sorted(set(text))
print(len(vocab))

65


In [55]:
char2id = {u:i for i, u in enumerate(vocab)}
id2char = np.array(vocab)
print(id2char)
text_as_int = [char2id[char] for char in text]
print(text_as_int[:100])

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49, 8, 0, 0, 13, 50, 50, 10, 0, 31, 54, 43, 39, 49, 6, 1, 57, 54, 43, 39, 49, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 37, 53, 59]


# 模型

In [56]:
# 设定每个输入句子长度的最大值
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(id2char[i.numpy()])


F
i
r
s
t


In [57]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(1):
  print(item.numpy())
  print("".join(id2char[item.numpy()]))



[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1]
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


In [58]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text
dataset = sequences.map(split_input_target)

In [59]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(id2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(id2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [60]:
# 批大小
BATCH_SIZE = 64

# 设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列，
# 所以它不会试图在内存中重新排列整个序列。相反，
# 它维持一个缓冲区，在缓冲区重新排列元素。） 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [61]:
# 词集的长度
vocab_size = len(vocab)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024

In [67]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                    batch_input_shape=[BATCH_SIZE, None]))
model.add(tf.keras.layers.GRU(rnn_units,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer="glorot_uniform"))
model.add(tf.keras.layers.Dense(vocab_size))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_6 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_6 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [68]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(target_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

tf.Tensor(
[[47 52  1 ... 43 56 43]
 [ 2  0  0 ... 39 50 50]
 [43  1 41 ... 43 52 58]
 ...
 [60 43 52 ... 57  1 47]
 [60 43  1 ... 50 63  1]
 [25 43 57 ... 53 61  6]], shape=(64, 100), dtype=int32)
(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [69]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [ ]:
model.fit(dataset, epochs=100)
model.save_weights("rnn.model")

Epoch 1/100
172/172 [==============================] - 10s 59ms/step - loss: 3.3064 - accuracy: 0.1054
Epoch 2/100
172/172 [==============================] - 10s 60ms/step - loss: 2.8604 - accuracy: 0.2026
Epoch 3/100
172/172 [==============================] - 10s 59ms/step - loss: 3.0179 - accuracy: 0.1046
Epoch 4/100
172/172 [==============================] - 10s 57ms/step - loss: 3.0549 - accuracy: 0.0884
Epoch 5/100
172/172 [==============================] - 10s 57ms/step - loss: 3.0443 - accuracy: 0.0546
Epoch 6/100
172/172 [==============================] - 10s 57ms/step - loss: 2.9213 - accuracy: 0.1107
Epoch 7/100
172/172 [==============================] - 10s 57ms/step - loss: 2.9582 - accuracy: 0.0548
Epoch 8/100
 42/172 [======>.......................] - ETA: 7s - loss: 2.8681 - accuracy: 0.0589

In [ ]:
def generate_text(model, start_string):
  # 评估步骤（用学习过的模型生成文本）

  # 要生成的字符个数
  num_generate = 1000

  # 将起始字符串转换为数字（向量化）
  input_eval = [char2id[s] for s in start_string]
  print(input_eval)
  input_eval = tf.expand_dims(input_eval, 0)
  print(input_eval)

  # 空字符串用于存储结果
  text_generated = []

  # 低温度会生成更可预测的文本
  # 较高温度会生成更令人惊讶的文本
  # 可以通过试验以找到最好的设定
  temperature = 1.0

  # 这里批大小为 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # 删除批次的维度
      predictions = tf.squeeze(predictions, 0)

      # 用分类分布预测模型返回的字符
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(id2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                    batch_input_shape=[1, None]))
model.add(tf.keras.layers.GRU(rnn_units,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer="glorot_uniform"))
model.add(tf.keras.layers.Dense(vocab_size))
model.summary()
model.load_weights("rnn.model")
print(generate_text(model, start_string=u"ROMEO: "))